In [68]:
# Scraping Multiple images of Cats, Dogs, and whatever you want...
# so its time to create our own dataset by downloading images.

In [69]:
# for requirements 
!pip install -r requirements.txt

## Download chrome driver according to your chrome browser 

Link - https://chromedriver.chromium.org/downloads

In [70]:
import selenium
import time
from selenium import webdriver
import io
from io import StringIO
import string
from PIL import Image
import hashlib
import os
import requests

DRIVER_PATH = 'chromedriver.exe'
# make sure you put chromedriver exe file in same folder as where is your py file (Important)
# make sure you put chromedriver exe file in same folder as where is your py file (Important)
# my chrome browser Version 93.0.4577.63 (Official Build) (64-bit) this diver is work for this only,
# so you have to check your version and download the driver first.
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [71]:
wd.get('https://google.com')

In [72]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Pandas')

In [73]:
wd.quit() # closing the driver

In [74]:
# for download the image 
# !pip install Pillow

In [75]:
# def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
def fetch_image_urls(query:'tortoise', max_links_to_fetch:5, wd:'instantiated Webdriver', sleep_between_interactions:int=1):

    # query : Search term, like Dog
    # max_links_to_fetch : Number of links the scraper is supposed to collect
    # webdriver : instantiated Webdriver
    
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [76]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [77]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=10):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [78]:
search_term = 'tortoise'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)


Found: 100 search results. Extracting links from 0:100
Found: 10 image links, done!
SUCCESS - saved https://res.cloudinary.com/dk-find-out/image/upload/q_80,w_1920,f_auto/DCTM_Penguin_UK_DK_AL345587_aoxfca2_ebtgry.jpg - as ./images\tortoise\35c376835d.jpg
SUCCESS - saved https://a-z-animals.com/media/2019/11/Tortoise-radiated.jpg - as ./images\tortoise\2b336787f4.jpg
SUCCESS - saved https://i.natgeofe.com/n/db6b3131-73af-404a-a3ca-51c6f15286c3/rare-tortoise-missing_3x4.jpg - as ./images\tortoise\1e449f34b3.jpg
SUCCESS - saved https://m.media-amazon.com/images/I/81pnPeU7dYL._SL1500_.jpg - as ./images\tortoise\c18f746b9d.jpg
SUCCESS - saved https://helios-i.mashable.com/imagery/articles/04if4A1xs7Q0purDN53SN1S/hero-image.fill.size_1248x702.v1630148708.png - as ./images\tortoise\3bda674e09.jpg
SUCCESS - saved https://cdn.britannica.com/25/178225-050-AAF9B658/Desert-tortoise.jpg - as ./images\tortoise\f0d5f60fce.jpg
SUCCESS - saved https://files.worldwildlife.org/wwfcmsprod/images/HERO_Gia